In [1]:
import os
os.chdir("../")

### Model

In [2]:
from rag_pipeline.components.models import ModelConfig

In [3]:
model = ModelConfig()

In [4]:
model.get_agent_model(
    agent_name = "question_rewriter"
).get("name")

'gpt-4o-mini'

### Prompts

In [5]:
from rag_pipeline.components.prompts import render_prompt

In [7]:
render_prompt(
    prompt_name = "question_rewriter",
    current_question = "What is RAG?",
    conversation = "User: What is AI?\nAgent: AI stands for Artificial Intelligence"
)

{'system': 'ROLE\n- You are a question rewriter for a Retrieval-Augmented Generation system.\n\nINPUTS\n- "current_question": The user\'s most recent query.\n- "conversation": A short history of recent interactions (if available).\n\nTASK\n- Rephrase the query into a clear, standalone form that captures the full intent of the user\'s question.\n\nRULES\n- Use conversation only to clarify vague references like "this file" or "the previous report".\n- Keep the question concise and factual (under 40 tokens).\n- Do not guess missing details; preserve ambiguity if uncertain.\n- Avoid multi-sentence questions.',
 'user': 'USER QUERY: What is RAG?\nCHAT HISTORY: User: What is AI?\nAgent: AI stands for Artificial Intelligence',
 'output_schema': '{\n  "rephrased_question": "string"\n}\n'}

### Graph

In [2]:
from graph import run_graph

In [3]:
graph = run_graph()

In [4]:
user_input = "What is RAG?"
input_data = {
    "question": user_input
}

In [5]:
await graph.ainvoke(
    input = input_data,
    config = {
        "configurable": {
            "retriever": None
        }
    }
)

{'question': 'What is RAG?',
 'rephrased_question': 'What does RAG stand for and what does it mean?'}